<a href="https://colab.research.google.com/github/petrucior/verilog/blob/main/processador_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de processador simplificado

Author: Petrúcio Medeiros

- Descrição: Neste projeto foi considerado um contador de programa (PC), uma memória de instruções que contém 100 endereços e cada endereço com 32 bits. Um banco de registradores com 32 endereços de 32 bits previamente definido com os endereços 0 e 1 preenchidos com os valores 3 e 5, respectivamente. Uma unidade lógica aritmética (ULA) para realizar a operação de soma. Finalmente, uma Unidade Central de Processamento (CPU) para coordenar as operações entre essas unidades.

  As instruções são codificadas como no MIPS 32:
  - Opcode : 6 bits
  - Registrador destino : 5 bits
  - Registrador fonte 1 : 5 bits
  - Registrador fonte 2 : 5 bits
  - Shamp ( operações de deslocamento ) : 5 bits
  - Funct ( variacoes das operacoes especificadas do opcode ) : 6 bits

In [1]:
# biblioteca do verilog
%%bash
sudo apt-get update
sudo apt-get install verilog gtkwave

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,733 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 https://cloud

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 14.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [2]:
%%file pc.sv
// Contador de programa ( PC )
module pc (clk, address);
  // Declaracao de portas
  input clk;
  reg [31:0] counter;
  output [31:0] address;

  // Iniciando endereco
  initial begin
    counter = -32'd1;
  end

  // Funcionamento do contador => contador + 1
  always @(posedge clk) begin
    counter <= counter + 32'd1;
  end

  // Atribuindo o valor do endereco
  assign address = counter;
endmodule

Writing pc.sv


**Instruções:**

----

**add rd, r1, r2;**

* 6 bits ( opcode ) = **000001**
* 5 bits ( registrador destino ) = **00010**
* 5 bits ( registrador origem 1 ) = **00000**
* 5 bits ( registrador origem 2 ) = **00001**
* 5 bits ( shamt - operacoes de deslocamento ) = **00000**
* 6 bits ( funct - variacoes das operacoes especificadas do opcode ) = **00000**

```
Ex.: add $s2, $s0, $s1;
         $s2 = 3 + 5 = 8
```

---

**sub rd, r1, r2;**

* 6 bits ( opcode ) = **000010**
* 5 bits ( registrador destino ) = **00011**
* 5 bits ( registrador origem 1 ) = **00001**
* 5 bits ( registrador origem 2 ) = **00000**
* 5 bits ( shamt - operacoes de deslocamento ) = **00000**
* 6 bits ( funct - variacoes das operacoes especificadas do opcode ) = **00000**

```
Ex. sub $s3, $s1, $s0;
        $s3 = 5 - 3 = 2
```

In [3]:
%%file instructions.txt
00000100010000000000100000000000
00001000011000010000000000000000

Writing instructions.txt


In [4]:
%%file instructions_memory.sv
`define NULL 0

// Memória de instrucoes
module instructions_memory ( clk, programcounter, output_instruction );
  // Declaracao de portas
  input clk;
  input [31:0] programcounter;
  output [31:0] output_instruction;

  integer f, r; // arquivo
  reg [31:0] data = 32'd0;
  integer counterMemory = 32'd0;

  // Criando uma memoria de instrucoes com 100 enderecos de 32 bits
  reg [31:0] memoriaInstrucoes [99:0];

  // Iniciando a memoria de instrucoes
  /*
  initial begin
    // 6 bits ( opcode ) = 000001
    // 5 bits ( registrador destino ) = 00010
    // 5 bits ( registrador origem 1 ) = 00000
    // 5 bits ( registrador origem 2 ) = 00001
    // 5 bits ( shamt - operacoes de deslocamento ) = 00000
    // 6 bits ( funct - variacoes das operacoes especificadas do opcode ) = 00000
    memoriaInstrucoes[32'd0] = 32'b00000100010000000000100000000000;
  end
  */
  
  initial begin
    // Abrindo arquivo
    f = $fopen("/content/instructions.txt","r");
    // Se arquivo nao contem nenhuma informacao
    if ( f == `NULL ) begin
      $display("Arquivo sem instrucoes");
      $finish;
    end 

    while (! $feof( f )) begin
      r = $fscanf( f,"%32b\n", data);
      memoriaInstrucoes[counterMemory] = data;
      counterMemory = counterMemory + 32'd1;
    end

    // Fechando arquivo
    $fclose( f );
  end
  

  // Atribuindo o valor da instrucao
  assign output_instruction = memoriaInstrucoes[ programcounter ];
endmodule

Writing instructions_memory.sv


**Dados**: Cada linha do banco de registradores contém um dado.

```
| 32 bits |
 --------------------------------------
|    3    |    5    |  . . .  |        |  
 --------------------------------------
            100 endereços
```

In [5]:
%%file registers.txt
3
5

Writing registers.txt


In [6]:
%%file register_bank.sv
`define NULL 0

// Banco de registradores
module register_bank ( clk, addr_regd, addr_reg1, addr_reg2, data_in, value_regd, value_reg1, value_reg2 );
  // Declaracao de portas
  input clk;
  input [4:0] addr_regd, addr_reg1, addr_reg2;
  input [31:0] data_in;
  output [31:0] value_regd, value_reg1, value_reg2;

  integer f, r; // arquivo
  reg [31:0] data = 32'd0;
  integer counterRegister = 5'd0;

  // Criando um banco de registradores com 32 enderecos de 32 bits
  reg [31:0] registerBank [31:0];

  // Iniciar valores para os registradores reg1 e reg2 no banco de registradores
  /*
  initial begin
    registerBank[5'd0] = 32'd3;
    registerBank[5'd1] = 32'd5;
  end
  */

  initial begin
    // Abrindo arquivo
    f = $fopen("/content/registers.txt","r");
    // Se arquivo nao contem nenhuma informacao
    if ( f == `NULL ) begin
      $display("Arquivo sem dados");
      $finish;
    end 

    while (! $feof( f )) begin
      r = $fscanf( f,"%32d\n", data);
      registerBank[counterRegister] = data;
      counterRegister = counterRegister + 5'd1;
    end

    // Fechando arquivo
    $fclose( f );
  end


  // Descrevendo o comportamento de escrita no banco de registradores
  always @(posedge clk) begin
    registerBank[addr_regd] <= data_in; 
  end

  // Atribuicao de valores
  assign value_regd = registerBank[addr_regd];
  assign value_reg1 = registerBank[addr_reg1];
  assign value_reg2 = registerBank[addr_reg2];
endmodule

Writing register_bank.sv


In [7]:
%%file ula.sv
// Unidade logica aritmetica (ULA)
module ula ( opcode, in1, in2, out );
  // Declaracao de portas
  input [5:0] opcode;
  input [31:0] in1, in2;
  output [31:0] out;
  
  // Variavel para guardar resultado
  reg [31:0] result;

  // Descrevendo o comportamento da ULA
  always @(in1, in2, opcode) begin
    case ({opcode})
      6'd1 : result = in1 + in2;
      6'd2 : result = in1 - in2;
      default : result = 32'd0;
    endcase
  end

  // Atribuindo o valor do resultado
  assign out = result;
endmodule

Writing ula.sv


In [8]:
%%file cpu.v

`include "pc.sv"
`include "instructions_memory.sv"
`include "register_bank.sv"
`include "ula.sv"

// Unidade Central de Processamento
module cpu ( clock, value_output );
  // Declaracao de portas
  input clock;
  output [31:0] value_output;

  // Variaveis intermediarias
  wire [31:0] address, instruction, data_ula, value_regd, value_reg1, value_reg2;

  // PC
  pc p(clock, address);

  wire flag = 1'd1;

  // Acessando memoria de instrucoes
  instructions_memory im( clock, address, instruction );

  wire [4:0] regd, reg1, reg2;
  assign regd = instruction[25:21];
  assign reg1 = instruction[20:16];
  assign reg2 = instruction[15:11];

  wire [5:0] op;
  assign op = instruction[31:26];

  // Acessando banco de registradores
  register_bank rb( clock, regd, reg1, reg2, data_ula, value_regd, value_reg1, value_reg2 );

  // Operando sobre os valores dos registradores
  ula alu( op, value_reg1, value_reg2, data_ula );

  assign value_output = data_ula;

  /*

  // Atribuindo o valor do registrador destino
  assign value_output = value_regd;
  */

endmodule

/*

 Módulo de teste

*/
module teste(); // Definindo um módulo de teste, onde adiciono valores
parameter nbits = 8; // 2^{entradas} = 2^{3} = 8
reg counter; // Criando um registro de 2 bits para entradas
wire [31:0] z; // Declarando um fio de saída
integer k; // Declarando um inteiro para percorrer todas as possibilidades
	cpu t(counter, z);
	initial begin // // Início do bloco de comandos
		$display("clk | z"); // printf apenas com string
		$monitor(" %1b  | %1d", counter, z[31:0]); // printf passando variáveis
    counter = 0;
    /*
    Percorre a quantidade de saídas e armazena no contador
    */
		for (k=1; k<nbits; k=k+1)
     #1 counter = k;
		$finish;
	end // Fim do bloco de comandos
endmodule


Writing cpu.v


In [9]:
%%bash
iverilog cpu.v -o cpu
vvp cpu

clk | z
 0  | x
 1  | 8
 0  | 8
 1  | 2
 0  | 2
 1  | 0
 0  | 0
 1  | 0
